<a href="https://colab.research.google.com/github/JabiliSree/AA/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())  # Flatten the 3D data to 1D
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 89ms/step - accuracy: 0.3252 - loss: 1.8200 - val_accuracy: 0.4996 - val_loss: 1.3971
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 77s 83ms/step - accuracy: 0.5553 - loss: 1.2538 - val_accuracy: 0.5993 - val_loss: 1.1442
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 82ms/step - accuracy: 0.6137 - loss: 1.0955 - val_accuracy: 0.6071 - val_loss: 1.1152
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 83s 84ms/step - accuracy: 0.6528 - loss: 0.9893 - val_accuracy: 0.6457 - val_loss: 1.0077
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 83ms/step - accuracy: 0.6797 - loss: 0.9116 - val_accuracy: 0.6797 - val_loss: 0.9355
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 81ms/step - accuracy: 0.7042 - loss: 0.8497 - val_accuracy: 0.6762 - val_loss: 0.9130
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 65s 83ms/step - accuracy: 0.7248 - loss: 0.7871 - val_accuracy: 0.6998 - val_loss: 0.8698
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 85ms/step - accuracy: 0.7386 - loss: 0.7457 - 

In [ ]:
import numpy as np

# Load the MNIST dataset
from keras.datasets import mnist  # Keep for dataset loading
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
img_rows, img_cols = 28, 28
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0  # Flatten and normalize
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0
y_train = np.eye(10)[y_train]  # One-hot encode labels
y_test = np.eye(10)[y_test]

# Define neural network structure
input_size = img_rows * img_cols
hidden_size = 128
output_size = 10

# Initialize weights and biases
np.random.seed(0)
W1 = np.random.randn(input_size, hidden_size) * 0.01  # Input to hidden
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01  # Hidden to output
b2 = np.zeros((1, output_size))

def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)


def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + 1e-8)) / m
    return loss


def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    cache = (Z1, A1, Z2, A2)
    return A2, cache


def backward_propagation(X, y_true, cache):
    Z1, A1, Z2, A2 = cache
    m = X.shape[0]

    dZ2 = A2 - y_true
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * (Z1 > 0)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients

def update_parameters(gradients, learning_rate=0.01):
    global W1, b1, W2, b2
    W1 -= learning_rate * gradients["dW1"]
    b1 -= learning_rate * gradients["db1"]
    W2 -= learning_rate * gradients["dW2"]
    b2 -= learning_rate * gradients["db2"]


epochs = 12
batch_size = 128
learning_rate = 0.01

for epoch in range(epochs):
    permutation = np.random.permutation(x_train.shape[0])
    x_train_shuffled = x_train[permutation]
    y_train_shuffled = y_train[permutation]

    for i in range(0, x_train.shape[0], batch_size):
        X_batch = x_train_shuffled[i:i + batch_size]
        y_batch = y_train_shuffled[i:i + batch_size]


        y_pred, cache = forward_propagation(X_batch)


        loss = compute_loss(y_batch, y_pred)


        gradients = backward_propagation(X_batch, y_batch, cache)

        update_parameters(gradients, learning_rate)

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")

y_pred_test, _ = forward_propagation(x_test)
accuracy = np.mean(np.argmax(y_pred_test, axis=1) == np.argmax(y_test, axis=1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/12, Loss: 2.0163838755265746
Epoch 2/12, Loss: 0.9243883642628719
Epoch 3/12, Loss: 0.6144868623603666
Epoch 4/12, Loss: 0.4771324992696265
Epoch 5/12, Loss: 0.4873274247057939
Epoch 6/12, Loss: 0.5715835026055163
Epoch 7/12, Loss: 0.31621481243045024
Epoch 8/12, Loss: 0.4018104819224308
Epoch 9/12, Loss: 0.4461538720639369
Epoch 10/12, Loss: 0.23430398637446945
Epoch 11/12, Loss: 0.45801338057748103
Epoch 12/12, Loss: 0.2464558854451697
Test Accuracy: 91.16%
